<a href="https://colab.research.google.com/github/sanaaria/Ann/blob/main/wikipediaperRnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn

import numpy as np
import torch.nn.functional as F

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
with open('/content/drive/MyDrive/Persian-WikiText-1.txt/wikipediaperRnn.txt', 'r') as f:
    text = f.read()

In [4]:
text[:1000]

'\ufeffعنوان مقاله: صفحهٔ اصلی\n\n\n<templatestyles src="صفحه اصلی/\n\n\n\n\n\n\nعنوان مقاله: ویکی پدیا\n\n\nویکی پدیا (کوته نوشت به صورت «وپ» و «WP») یک دانشنامه برخط چندزبانه مبتنی بر وب با محتوای آزاد و همکاری باز است که با همکاری افراد داوطلب نوشته می شود و هر کسی که به اینترنت و وب دسترسی داشته باشد می تواند مقالات آن را ببیند و ویرایش کند. نام ویکی پدیا واژه ای ترکیبی است که از واژه های ویکی (وبگاه مشارکتی) و اِنسایکلوپدیا (Encyclopedia) (دانشنامه یا دائرةالمعارف) گرفته شده است. هدف ویکی پدیا آفرینش و انتشار جهانی یک دانشنامه با محتوای آزاد به تمامی زبان های زندهٔ دنیا است.\n\n\nویکی پدیای انگلیسی در تاریخ ۱۵ ژانویه ۲۰۰۱ (۲۶ دی ۱۳۷۹) به صورت مکملی برای دانشنامهٔ تخصصی نیوپدیا نوشته شد. بنیان گذاران آن «جیمی ویلز» و «لری سنگر» هستند. هم اکنون بنیاد غیرانتفاعی ویکی مدیا پروژهٔ ویکی پدیا را پشتیبانی می کند. میزبان های اینترنتی اصلی این وبگاه در شهر تامپای فلوریدا هستند. همچنین میزبان های اضافی دیگری هم در شهرهای آمستردام و سئول به این وبگاه یاری می رسانند.\n\n\n"ویکی پدیا" از پایان 

In [6]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
encoded = np.array([char2int[ch] for ch in text])

In [7]:
def one_hot_encode(arr, n_labels):
    
   
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
  
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
 
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [8]:
test_seq = np.array([[0, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)
print(one_hot)

[[[1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [9]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
   
    n_batches = len(arr)//batch_size_total
    
    arr = arr[:n_batches * batch_size_total]
   
    arr = arr.reshape((batch_size, -1))
    
    
    for n in range(0, arr.shape[1], seq_length):
        
        x = arr[:, n:n+seq_length]
       
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [10]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')
    

Training on GPU!


In [11]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
      
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
      
        self.lstm=nn.LSTM(len(self.chars),n_hidden,n_layers,
                          dropout=drop_prob,batch_first=True)
        
        
        self.dropout=nn.Dropout(drop_prob)
        
       
        self.fc=nn.Linear(n_hidden,len(self.chars))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        
        r_output, hidden = self.lstm(x, hidden)
        
       
        out = self.dropout(r_output)
        
      
        out = out.contiguous().view(-1, self.n_hidden)
        
      
        out = self.fc(out)
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
       
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [12]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
  
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
      
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
          
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()
            
            h = tuple([each.data for each in h])
           
            net.zero_grad()
            
        
            output, h = net(inputs, h)
            
           
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
           
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
           
            if counter % print_every == 0:
                
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                  
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() 
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [17]:

n_hidden = 512
n_layers = 2
net = CharRNN(chars, n_hidden, n_layers)
print(net)
batch_size = 64
seq_length = 100
n_epochs =  10
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=100)

CharRNN(
  (lstm): LSTM(205, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=205, bias=True)
)
Epoch: 1/10... Step: 100... Loss: 3.1098... Val Loss: 3.0440
Epoch: 2/10... Step: 200... Loss: 2.9608... Val Loss: 2.8706
Epoch: 3/10... Step: 300... Loss: 2.6800... Val Loss: 2.6792
Epoch: 4/10... Step: 400... Loss: 2.5469... Val Loss: 2.5404
Epoch: 5/10... Step: 500... Loss: 2.4863... Val Loss: 2.4463
Epoch: 6/10... Step: 600... Loss: 2.3558... Val Loss: 2.3681
Epoch: 7/10... Step: 700... Loss: 2.3140... Val Loss: 2.3018
Epoch: 8/10... Step: 800... Loss: 2.2858... Val Loss: 2.2451
Epoch: 9/10... Step: 900... Loss: 2.2377... Val Loss: 2.1992
Epoch: 9/10... Step: 1000... Loss: 2.1404... Val Loss: 2.1516
Epoch: 10/10... Step: 1100... Loss: 2.1328... Val Loss: 2.1119


In [18]:

model_name = 'wikipediaperRnn.net'
checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}
with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [19]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
       
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
 
        h = tuple([each.data for each in h])
   
        out, h = net(inputs, h)
 
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu()
        
    
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
  
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
     
        return net.int2char[char], h
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() 
    

    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)
    chars.append(char)
    
   
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)
    return ''.join(chars)

In [21]:
print(sample(net, 500, prime='مهندسی مکانیک نخست در طی ', top_k=2))

مهندسی مکانیک نخست در طی سال ۱۹۰۰ به در این مورد می شود.


این است که در این این میانه است که این این استان می کند و به این این می شدند. در استان می شود.


در ایر این این می شدند. است. است که این استان می کند. در سال ۱۹۰۰ بر می شود و این استان به دورهٔ می شود.


از این اینشتین، این می کند که در است که این این مورد ایران با از این می کند که داران از موان به دول این میان با این این استان بازد.


این میان به دولت استان می شود که در اینشانه ای از این این مورد می شود. داران از انتام مارد از این است که در سال ۱۹۰۰ مالی با با ا
